In [53]:
import numpy as np
from pathlib import Path
from keras.models import load_model
from tensorflow.keras.preprocessing import image

##### Get the best model and COVID/NON-COVID images

In [54]:
model_path = Path(".").resolve().parent.joinpath("models/Experiment 4.h5")

covid_images = Path("F:").resolve().joinpath("Datasets/SARS-COV-2 Ct-Scan Dataset/COVID")
non_covid_images = Path("F:").resolve().joinpath("Datasets/SARS-COV-2 Ct-Scan Dataset/non-COVID")

In [55]:
model = load_model(model_path)

# Classes For CNN Model Predictions
classes = {0: "COVID-19", 1: "NON-COVID"}

##### Function to predict the class label of an image

In [56]:
def GetPrediction(img):
    img = image.load_img(img, target_size=(224, 224))
    img = image.img_to_array(img) / 255
    x = np.expand_dims(img, axis=0)
    prediction = (model.predict(x, batch_size=128) > 0.5).astype("int")
    return classes.get(prediction[0][0])

##### Test NON-COVID images

In [57]:
non_covid_results = []
for img in non_covid_images.glob("*"):
    non_covid_results.append(GetPrediction(img))

print(f"Number of NON-COVID images: {len(non_covid_results)}\n"
      f"Correctly identified as NON-COVID: {non_covid_results.count('NON-COVID')}\n"
      f"Incorrectly identified as COVID: {non_covid_results.count('COVID-19')}")

Number of NON-COVID images: 1229
Correctly identified as NON-COVID: 376
Incorrectly identified as COVID: 853


##### Test COVID images

In [58]:
covid_results = []
for img in covid_images.glob("*"):
    covid_results.append(GetPrediction(img))

print(f"Number of COVID images: {len(covid_results)}\n"
      f"Correctly identified as COVID: {covid_results.count('COVID-19')}\n"
      f"Incorrectly identified as NON-COVID: {covid_results.count('NON-COVID')}")

Number of COVID images: 1252
Correctly identified as COVID: 1050
Incorrectly identified as NON-COVID: 202
